## Question 4
`Write a query to retrieve the average revenue per customer for each country in the database, but only include countries with more than 3 customers. Make sure to check for duplicate customer info.`

(Hint: use a combination of subqueries and the HAVING clause)

## Expected Output

| Country   |   AvgRevenuePerCustomer |   TotalRevenue |
|:----------|------------------------:|---------------:|
| Germany   |                1.39714  |         156.48 |
| France    |                1.11486  |         195.09 |
| Brazil    |                1.08629  |         190.09 |
| Canada    |                0.678482 |         303.95 |
| USA       |                0.442147 |         523.06 |

In [1]:
%run ../utils/setup_notebook.ipynb

In [2]:
%%sql 

SELECT *
FROM invoices
LIMIT 1;

InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98


In [3]:
%%sql 

SELECT BillingCountry, 
    COUNT(BillingCountry) CountryCount,  
    TRUNCATE(AVG(Total),2) AverageRevenue,
    CEILING(SUM(Total)) TotalRevenue
FROM invoices
GROUP BY BillingCountry
HAVING COUNT(CustomerId) > 3
ORDER BY TotalRevenue DESC;

BillingCountry,CountryCount,AverageRevenue,TotalRevenue
USA,91,5.74,524.0
Canada,56,5.42,304.0
France,35,5.57,196.0
Brazil,35,5.43,191.0
Germany,28,5.58,157.0
United Kingdom,21,5.37,113.0
Czech Republic,14,6.44,91.0
Portugal,14,5.51,78.0
India,13,5.78,76.0
Chile,7,6.65,47.0


In [4]:
%%sql 
--  Create a temporary table with the results of the query
CREATE TEMPORARY TABLE temporary_results AS
SELECT BillingCountry, 
    COUNT(BillingCountry) CountryCount,  
    TRUNCATE(AVG(Total),2) AverageRevenue,
    CEILING(SUM(Total)) TotalRevenue
FROM invoices
GROUP BY BillingCountry
HAVING COUNT(CustomerId) > 3
ORDER BY TotalRevenue DESC;

SELECT * FROM temporary_results;

BillingCountry,CountryCount,AverageRevenue,TotalRevenue
USA,91,5.74,524.0
Canada,56,5.42,304.0
France,35,5.57,196.0
Brazil,35,5.43,191.0
Germany,28,5.58,157.0
United Kingdom,21,5.37,113.0
Czech Republic,14,6.44,91.0
Portugal,14,5.51,78.0
India,13,5.78,76.0
Chile,7,6.65,47.0


In [5]:
%%sql

SELECT BillingCountry, 
       COUNT(DISTINCT CustomerId) AS CustomerCount,
       TRUNCATE(SUM(Total) / COUNT(DISTINCT CustomerId), 2) AS AvgRevenuePerCustomer,
       CEILING(SUM(Total)) AS TotalRevenue
FROM invoices
GROUP BY BillingCountry
HAVING COUNT(DISTINCT CustomerId) > 3
ORDER BY AvgRevenuePerCustomer DESC;


BillingCountry,CustomerCount,AvgRevenuePerCustomer,TotalRevenue
USA,13,40.23,524.0
Germany,4,39.12,157.0
France,5,39.01,196.0
Brazil,5,38.01,191.0
Canada,8,37.99,304.0


In [6]:
%%sql

-- What is the average total per customer by country?

SELECT c.Country, 
    AVG(i.Total / t.CustomerCount) AS AvgRevenuePerCustomer,
    t.TotalRevenue
FROM customers c
JOIN (
    SELECT BillingCountry, COUNT(DISTINCT CustomerId) AS CustomerCount, TRUNCATE(SUM(Total), 2) AS TotalRevenue
    FROM invoices
    GROUP BY BillingCountry
    HAVING COUNT(DISTINCT CustomerId) > 3
) t ON c.Country = t.BillingCountry
JOIN invoices i ON c.CustomerId = i.CustomerId
GROUP BY c.Country
ORDER BY AvgRevenuePerCustomer DESC;


Country,AvgRevenuePerCustomer,TotalRevenue
Germany,1.397142857142857,156.48
France,1.1148571428571432,195.09
Brazil,1.0862857142857145,190.09
Canada,0.6784821428571428,303.95
USA,0.442147083685545,523.06


### Print the ***`Expected Output`*** for the question's markdown

In [7]:
import sys 
sys.path.append("..")

from utils.format_sql_result import format_sql_result

In [8]:
%%sql query_output << SELECT c.Country, 
    AVG(i.Total / t.CustomerCount) AS AvgRevenuePerCustomer,
    t.TotalRevenue
FROM customers c
JOIN (
    SELECT BillingCountry, COUNT(DISTINCT CustomerId) AS CustomerCount, TRUNCATE(SUM(Total), 2) AS TotalRevenue
    FROM invoices
    GROUP BY BillingCountry
    HAVING COUNT(DISTINCT CustomerId) > 3
) t ON c.Country = t.BillingCountry
JOIN invoices i ON c.CustomerId = i.CustomerId
GROUP BY c.Country
ORDER BY AvgRevenuePerCustomer DESC;

Returning data to local variable query_output


In [9]:
format_sql_result(query_output)

| Country   |   AvgRevenuePerCustomer |   TotalRevenue |
|:----------|------------------------:|---------------:|
| Germany   |                1.39714  |         156.48 |
| France    |                1.11486  |         195.09 |
| Brazil    |                1.08629  |         190.09 |
| Canada    |                0.678482 |         303.95 |
| USA       |                0.442147 |         523.06 |
